In [1]:
import os 
import glob
import cv2
import numpy as np
import radiomics
from SimpleITK import GetImageFromArray
from radiomics.featureextractor import RadiomicsFeatureExtractor # This module is used for interaction with pyradiomic
import logging
import pandas as pd
logging.getLogger('radiomics').setLevel(logging.CRITICAL + 1)  
import SimpleITK

In [2]:
thyroid_mask = sorted(glob.glob("mask/*"))

In [3]:
thyroid_img = sorted(glob.glob("img/*/*"))

In [4]:
df=pd.DataFrame()
df['mask'] = thyroid_mask
df['image'] = thyroid_img
df['ids'] = df['image'].str.split('/').str[-2]
ids_list= df['ids'].tolist()

In [7]:
params = os.path.abspath("radiomics_extractor.yaml")

In [16]:
###retrieve radiomics features per image
radiomics_all=pd.DataFrame()
image_list=[]
for j in ids_list:
    texture_extractor = RadiomicsFeatureExtractor(params)

    temp_data = df[df['ids']==j]
    temp_data = temp_data.reset_index(drop=True)
    sample_img=[]
    sample_mask=[]
    for idx in temp_data.shape[0]:
        img=cv2.imread(temp_data['image'][idx],0)
        sample_img.append(img)
        
        mask=cv2.imread(temp_data['mask'][idx],0)
        sample_mask.append(mask)
    
    img_datay=np.array(sample_mask).astype(int)
    img_datax = np.array(sample_img)
    
    for idx in range(img_datax.shape[0]):
        if img_datay[idx].sum()==0:
            continue
        results = texture_extractor.execute(GetImageFromArray(img_datax[idx]),
                                    GetImageFromArray(img_datay[idx])
                                    #GetImageFromArray((img_datay>0).astype(np.uint8))
                            )
        df1 = pd.DataFrame([results]).T
        radiomics_all=pd.concat([radiomics_all, df1], axis=1)
    image_list.extend(temp_data['image'].tolist())

radiomics_all.columns=image_list
radiomics_all=radiomics_all.T
radiomics_all['filename'] = radiomics_all.index

In [ ]:
### merge radiomics feature with clinical feature
df_clinic = pd.read_csv(clinical_path)
df_merge = df_clinic.merge(radiomics_all,on ='filename',how='inner')
df_merge['basename'] = df_merge['filename'].str.split('/').str[-2]